In [1]:
import os
import sys
from pyspark.sql.functions import date_format , split , col, unix_timestamp, from_unixtime, expr  ,when
from pyspark.sql import SparkSession
from hdfs import InsecureClient
import pandas as pd 


In [41]:
print("a")

a


In [2]:

def get_hdfs_csv_files(client, folder_path):
    csv_files = {}
    # Listing files in the folder
    file_status = client.list(folder_path)
    # Iterating through files
    for file_or_folder_name in file_status:
        file_or_folder_full_path = folder_path + '/' + file_or_folder_name
        if client.status(file_or_folder_full_path)['type'] == 'DIRECTORY':
            # If it's a directory, recursively call the function
            csv_files.update(get_hdfs_csv_files(client, file_or_folder_full_path))
        elif file_or_folder_name.endswith('.csv'):
            # If it's a CSV file, add it to the dictionary along with its name
            csv_files[file_or_folder_name] = file_or_folder_full_path
    return csv_files


def add_filename_column(df, filename):
    df['Filename'] = filename
    return df

def process_csv_files(csv_files_dict, hdfs_client):
    dfs_with_filenames = []
    for filename, filepath in csv_files_dict.items():
        with hdfs_client.read(filepath) as reader:
            try:
                df = pd.read_csv(reader)
                if not df.empty:
                    df = add_filename_column(df, filename)
                    dfs_with_filenames.append(df) 
                else:
                    print(f"Empty DataFrame found in file: {filename}")
            except pd.errors.EmptyDataError:
                print(f"Empty file: {filename}")
            except pd.errors.ParserError as e:
                print(f"Error parsing file {filename}: {e}")
    concatenated_df = pd.concat(dfs_with_filenames, ignore_index=True)
    return concatenated_df


# Arrival data 


In [3]:
# Example usage:
hdfs_url = 'http://localhost:50070'
folder_path = '/user/PFE_data/arrival_flights'  # Adjust this path accordingly

# Create an HDFS client
client = InsecureClient(hdfs_url)

# Get all CSV files from the specified folder and its subfolders
csv_files_dict = get_hdfs_csv_files(client, folder_path)
print()
# Process CSV files: open each CSV file, add a filename column, and store the DataFrame in a dictionary
dfs_arrival = process_csv_files(csv_files_dict, client)

dfs_arrival


Empty DataFrame found in file: Anguilla2024-03-13.csv
Empty DataFrame found in file: Antarctica2024-03-19.csv
Empty file: Cameroon2024-03-14.csv
Empty DataFrame found in file: El-Salvador2024-03-13.csv
Empty DataFrame found in file: Estonia2024-03-13.csv
Empty DataFrame found in file: Grenada2024-03-19.csv
Error parsing file Guinea2024-03-08.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 14

Empty file: Guyana2024-03-14.csv
Empty DataFrame found in file: Lebanon2024-03-19.csv
Empty file: Malaysia2024-03-14.csv
Empty DataFrame found in file: Montserrat2024-03-13.csv
Error parsing file Norway2024-03-07.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 14

Empty DataFrame found in file: Sao-Tome-And-Principe2024-03-19.csv
Empty DataFrame found in file: Uruguay2024-03-13.csv
Error parsing file Wallis-And-Futuna2024-03-07.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 14



,Date,Flight Time,Aircraft,Origin Airport,flight,Aircraft Model,Flight Status,time_arrival,Destination Aeroport,Filename
0,"Thursday, Mar 21",14:40,Go2Sky,"Bucharest Henri Coanda International Airport, ...",W43205,B738,Delayed,15:04,Tirana International Airport,Albania2024-03-21.csv
1,"Thursday, Mar 21",14:40,Wizz Air,"Ancona Airport, Italy",W43856,A321,Landed,14:30,Tirana International Airport,Albania2024-03-21.csv
2,"Thursday, Mar 21",14:45,Ryanair,"Bucharest Henri Coanda International Airport, ...",FR8415,B738,Landed,14:18,Tirana International Airport,Albania2024-03-21.csv
3,"Thursday, Mar 21",15:00,Pegasus,"Istanbul Sabiha Gokcen International Airport, ...",PC283,A320,Landed,14:37,Tirana International Airport,Albania2024-03-21.csv
4,"Thursday, Mar 21",15:15,Wizz Air,"Athens Eleftherios Venizelos, Greece",W44814,A321,Estimated,15:13,Tirana International Airport,Albania2024-03-21.csv
...,...,...,...,...,...,...,...,...,...,...
66827,"Saturday, Mar 09",19:20,Air France,"Paris Orly Airport, France",AF7566,320,Scheduled,Scheduled,NaN,example.csv
66828,"Saturday, Mar 09",21:10,Air Corsica,"Nice Cote d'Azur Airport, France",XK105,AT7,Scheduled,Scheduled,NaN,example.csv
66829,"Saturday, Mar 09",21:30,Air Corsica,"Marseille Provence Airport, France",XK157,32N,Scheduled,Scheduled,NaN,example.csv
66830,"Sunday, Mar 10",00:06,AlbaStar,"Aqaba King Hussein International Airport, Jordan",LAV1876,B738,Scheduled,Scheduled,NaN,example.csv


In [4]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Read from Hadoop") \
    .getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
#spark.conf.set("spark.memory.offHeap.enabled","true") 
#spark.conf.set("spark.memory.offHeap.size","10g")

# Set configurations 

arrival_data = spark.createDataFrame(dfs_arrival)


In [5]:
arrival_data.show()

+----------------+-----------+------------------+--------------------+------+--------------+-------------+------------+--------------------+--------------------+
|            Date|Flight Time|          Aircraft|      Origin Airport|flight|Aircraft Model|Flight Status|time_arrival|Destination Aeroport|            Filename|
+----------------+-----------+------------------+--------------------+------+--------------+-------------+------------+--------------------+--------------------+
|Thursday, Mar 21|      14:40|            Go2Sky|Bucharest Henri C...|W43205|          B738|      Delayed|       15:04|Tirana Internatio...|Albania2024-03-21...|
|Thursday, Mar 21|      14:40|          Wizz Air|Ancona Airport, I...|W43856|          A321|       Landed|       14:30|Tirana Internatio...|Albania2024-03-21...|
|Thursday, Mar 21|      14:45|           Ryanair|Bucharest Henri C...|FR8415|          B738|       Landed|       14:18|Tirana Internatio...|Albania2024-03-21...|
|Thursday, Mar 21|      15:0

In [20]:
from pyspark.sql.functions import when, create_map, lit, split, trim, regexp_extract, substring, concat, to_date, col
from itertools import chain  # Import chain function from itertools module
from pyspark.sql.types import TimestampType  # Add this import statement


def process_arrival_data(spark_df):
    # Drop rows with null values
    df = spark_df.na.drop()

    # Split the "date" column into "day", "month", and "month_number" columns
    df = df.withColumn("day", split(df["date"], ",")[0])
    df = df.withColumn("month", trim(split(df["date"], ",")[1]))
    df = df.withColumn("day_number", trim(split(df["date"], " ")[2]))
    df = df.withColumn('Origin Country', split(df['Origin Airport'], ',')[1])

    # Define a mapping dictionary for month names
    month_mapping = {
        "Jan": "January", "Feb": "February", "Mar": "March", "Apr": "April", 
        "May": "May", "Jun": "June", "Jul": "July", "Aug": "August", 
        "Sep": "September", "Oct": "October", "Nov": "November", "Dec": "December"
    }

    # Convert the dictionary to a map
    mapping_expr = create_map([lit(x) for x in chain(*month_mapping.items())])

    # Extract the first character of the "month" column
    first_word_month = split(col("month"), " ")[0]

    # Apply the mapping to the first character
    df = df.withColumn("complete_month", mapping_expr.getItem(first_word_month))

    # Drop the original "month" column
    df = df.drop("month")
    df = df.drop("Date")


    # Remove duplicate rows
    df = df.dropDuplicates()

    # Extract "date_extraction" column based on conditions
    df = df.withColumn("date_extraction", 
                       when(df["filename"].rlike("\\d"), 
                            substring(df["filename"], -14, 10))
                       .otherwise(None))

    # Rename the "Aircraft" column to "Airline"
    df = df.withColumnRenamed("Aircraft", "Airline")
    df = df.withColumnRenamed("Destination Aeroport", "Destination Airport")


    # Add a new column "Airport country" based on filename content
    df = df.withColumn("Airport country",
                       when(df["Filename"].contains("_"),
                            split(col("Filename"), "_")[0])
                       .otherwise(regexp_extract(col("Filename"), "^(.*?)(?=[0-9])", 1)))
    df = df.drop("Filename")
    
    # Add a column "year" with constant value 2024
    df = df.withColumn("year", lit(2024))
    df = df.withColumn("Flight Time2", unix_timestamp(col("Flight Time"), "HH:mm").cast(TimestampType()))

    df = df.withColumn("time_arrival2", 
                       when(col("time_arrival").rlike("\\d"), 
                            unix_timestamp(col("time_arrival"), "HH:mm").cast(TimestampType()))
                       .otherwise(None))
    df = df.withColumn("delay", 
                               when(col("time_arrival2").isNotNull(), 
                                    expr("time_arrival2 - `Flight Time2`"))
                               .when(col("time_arrival") == "Scheduled", "Scheduled")
                               .when(col("time_arrival") == "Unknown", "Unknown")
                               .when(col("time_arrival") == "Canceled", "Canceled")
                               .otherwise("No"))
    df = df.withColumn("delay",
                               expr("CASE WHEN delay LIKE 'INTERVAL%' THEN regexp_extract(delay, '([0-9]+:[0-9]+:[0-9]+)', 1) ELSE delay END"))
    df =  df.drop("time_arrival2")
    df =  df.drop("Flight Time2")
    # Concatenate month, day, and year columns to create a date string
    df = df.withColumn("date_string", concat(col("complete_month"), lit("-"), col("day_number"), lit("-"), col("year")))
    # Convert the date string to a date type
    df = df.withColumn("date", to_date(col("date_string"), "MMMM-dd-yyyy"))
    df=df.drop("date_string")

    return df



In [21]:
# Apply the function to your DataFrame
arrival_data_processed = process_arrival_data(arrival_data)

arrival_data_processed.toPandas()

C:\Users\USER\anaconda3333\lib\site-packages\pyspark\sql\column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


,Flight Time,Airline,Origin Airport,flight,Aircraft Model,Flight Status,time_arrival,Destination Airport,day,day_number,Origin Country,complete_month,date_extraction,Airport country,year,delay,date
0,15:10,St Barth Commuter,"St. Jean Gustaf III Airport, Guadeloupe",PV49,C208,Landed,15:17,Antigua V. C. Bird International Airport,Monday,18,Guadeloupe,March,2024-03-19,Antigua-And-Barbuda,2024,00:07:00,2024-03-18
1,13:55,Bahamasair,Fort Lauderdale Hollywood International Airpor...,UP142,AT76,Scheduled,Scheduled,Freeport Grand Bahama International Airport,Wednesday,06,United States,March,None,Bahamas,2024,Scheduled,2024-03-06
2,19:30,Bahamasair,"Freeport Grand Bahama International Airport, B...",UP326,AT46,Scheduled,Scheduled,Nassau Lynden Pindling International Airport,Wednesday,06,Bahamas,March,None,Bahamas,2024,Scheduled,2024-03-06
3,11:22,American Eagle,"Miami International Airport, United States",AA3726,E75L,Scheduled,Scheduled,North Eleuthera Airport,Wednesday,06,United States,March,None,Bahamas,2024,Scheduled,2024-03-06
4,12:55,Gulf Air,"Doha Hamad International Airport, Qatar",GF525,A320,Scheduled,Scheduled,Bahrain International Airport,Tuesday,12,Qatar,March,2024-03-11,Bahrain,2024,Scheduled,2024-03-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58525,20:50,VietJet Air,"Ho Chi Minh City International Airport, Vietnam",VJ254,320,Scheduled,Scheduled,Sao Vang Tho Xuan Airport,Wednesday,06,Vietnam,March,None,Vietnam,2024,Scheduled,2024-03-06
58526,15:50,Vietnam Airlines,"Ho Chi Minh City International Airport, Vietnam",VN1266,A321,Delayed,16:12,Vinh Airport,Wednesday,06,Vietnam,March,None,Vietnam,2024,00:22:00,2024-03-06
58527,08:30,VietJet Air,"Ho Chi Minh City International Airport, Vietnam",VJ210,320,Scheduled,Scheduled,Vinh Airport,Thursday,07,Vietnam,March,None,Vietnam,2024,Scheduled,2024-03-07
58528,15:35,InterCaribbean Airways,Santo Domingo Las Americas International Airpo...,JY411,E120,Scheduled,Scheduled,Beef Island Terrance B. Lettsome Airport,Monday,18,Dominican Republic,March,2024-03-18,Virgin-Islands-British,2024,Scheduled,2024-03-18


# Departure data


In [22]:
# Example usage:
hdfs_url = 'http://localhost:50070'
folder_path = '/user/PFE_data/departure_flights'  # Adjust this path accordingly

# Create an HDFS client
client = InsecureClient(hdfs_url)

# Get all CSV files from the specified folder and its subfolders
csv_files_dict = get_hdfs_csv_files(client, folder_path)
print()
# Process CSV files: open each CSV file, add a filename column, and store the DataFrame in a dictionary
dfs_departure = process_csv_files(csv_files_dict, client)

dfs_departure


Error parsing file Cayman-Islands2024-03-07.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 20

Error parsing file Congo2024-03-07.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 20

Error parsing file Dominican-Republic2024-03-07.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 20

Empty DataFrame found in file: Grenada2024-03-19.csv
Empty DataFrame found in file: Montenegro2024-03-12.csv
Empty file: Solomon-Islands2024-03-14.csv
Empty file: Tajikistan2024-03-14.csv
Empty file: Timor-leste-east-Timor2024-03-14.csv
Empty file: Trinidad-And-Tobago2024-03-14.csv


,Date,Flight Time,Airline,Destination Aeroport,flight,Aircraft Model,Flight Status,time_arrival,Original Aeroport,Filename,Aircraft
0,"Sunday, Mar 10",23:05,Wizz Air,"London Luton Airport, United Kingdom",W94472,A21N,Departed,03:14,Tirana International Airport,Albania2024-03-12.csv,NaN
1,"Monday, Mar 11",01:55,Pegasus,"Istanbul Sabiha Gokcen International Airport, ...",PC282,A21N,Departed,01:57,Tirana International Airport,Albania2024-03-12.csv,NaN
2,"Monday, Mar 11",04:00,AirSERBIA,"Belgrade Nikola Tesla Airport, Serbia",JU217,AT76,Departed,04:01,Tirana International Airport,Albania2024-03-12.csv,NaN
3,"Monday, Mar 11",04:55,Austrian Airlines,"Vienna International Airport, Austria",OS850,A320,Departed,04:55,Tirana International Airport,Albania2024-03-12.csv,NaN
4,"Monday, Mar 11",06:00,Wizz Air,"Dortmund Airport, Germany",W43841,A320,Departed,06:02,Tirana International Airport,Albania2024-03-12.csv,NaN
...,...,...,...,...,...,...,...,...,...,...,...
50093,"Saturday, Mar 09",13:10,Air Tanzania,"Lubumbashi International Airport, Democratic R...",TC213,223,Scheduled,Scheduled,Ndola Simon Mwansa Kapwepwe International Airport,Zambia2024-03-08.csv,NaN
50094,"Saturday, Mar 09",13:15,Airlink,"Johannesburg OR Tambo International Airport, S...",4Z151,E135,Scheduled,Scheduled,Ndola Simon Mwansa Kapwepwe International Airport,Zambia2024-03-08.csv,NaN
50095,"Saturday, Mar 09",13:45,Proflight Zambia,"Lusaka Kenneth Kaunda International Airport, Z...",P0335,J41,Scheduled,Scheduled,Ndola Simon Mwansa Kapwepwe International Airport,Zambia2024-03-08.csv,NaN
50096,"Saturday, Mar 09",18:15,Proflight Zambia,"Lusaka Kenneth Kaunda International Airport, Z...",P0309,J41,Scheduled,Scheduled,Ndola Simon Mwansa Kapwepwe International Airport,Zambia2024-03-08.csv,NaN


In [23]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Read from Hadoop") \
    .getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
#spark.conf.set("spark.memory.offHeap.enabled","true") 
#spark.conf.set("spark.memory.offHeap.size","10g")

# Set configurations

departure_data = spark.createDataFrame(dfs_departure)


C:\Users\USER\anaconda3333\lib\site-packages\pyspark\sql\pandas\conversion.py:351: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Expected bytes, got a 'int' object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


In [11]:
from pyspark.sql.functions import when, create_map, lit, split, trim, regexp_extract, substring, concat, to_date, col

from itertools import chain
from pyspark.sql.functions import expr, when, col, unix_timestamp
from pyspark.sql.types import TimestampType

def process_departure_data(departure_data):
    # Remove rows with null values
    df = departure_data.na.drop()

    # Split the "date" column into "day", "month", and "month_number" columns
    df = df.withColumn("day", split(df["date"], ",")[0])
    df = df.withColumn("month", trim(split(df["date"], ",")[1]))
    df = df.withColumn("day_number", trim(split(df["date"], " ")[2]))
    df = df.drop("Aircraft")

    # Define a mapping dictionary for month names
    month_mapping = {
        "Jan": "January", "Feb": "February", "Mar": "March", "Apr": "April", 
        "May": "May", "Jun": "June", "Jul": "July", "Aug": "August", 
        "Sep": "September", "Oct": "October", "Nov": "November", "Dec": "December"
    }

    # Convert the dictionary to a map
    mapping_expr = create_map([lit(x) for x in chain(*month_mapping.items())])

    # Extract the first character of the "month" column
    first_word_month = split(col("month"), " ")[0]

    # Apply the mapping to the first character
    df = df.withColumn("complete_month", mapping_expr.getItem(first_word_month))

    # Drop the original "month" column
    df = df.drop("month")

    # Remove duplicate rows
    df = df.dropDuplicates()

    # Add a new column "Airport country" based on filename content
    df = df.withColumn("Airport country",
                       when(df["Filename"].contains("_"),
                            split(col("Filename"), "_")[0])
                       .otherwise(regexp_extract(col("Filename"), "^(.*?)(?=[0-9])", 1)))
    df = df.drop("Filename")

    # Rename columns
    df = df.withColumnRenamed("Original Aeroport", "Origin Airport")
    df = df.withColumnRenamed("Destination Aeroport", "Destination Airport")

    # Split "Destination Airport" column to get "Destination Country"
    df = df.withColumn('Destination Country', split(df['Destination Airport'], ',')[1])
    
    # Add a column "year" with constant value 2024
    df = df.withColumn("year", lit(2024))
        # Convert string column "Flight Time" to timestamp type
    df = df.withColumn("Flight Time2", unix_timestamp(col("Flight Time"), "HH:mm").cast(TimestampType()))
    # Convert "arrival_time" column to timestamp type for HH:mm values
    df = df.withColumn("time_arrival2", 
                       when(col("time_arrival").rlike("\\d"), 
                            unix_timestamp(col("time_arrival"), "HH:mm").cast(TimestampType()))
                       .otherwise(None))
    df = df.withColumn("delay", 
                               when(col("time_arrival2").isNotNull(), 
                                    expr("time_arrival2 - `Flight Time2`"))
                               .when(col("time_arrival") == "Scheduled", "Scheduled")
                               .when(col("time_arrival") == "Unknown", "Unknown")
                               .when(col("time_arrival") == "Canceled", "Canceled")
                               .otherwise("No"))
    df = df.withColumn("delay",
                               expr("CASE WHEN delay LIKE 'INTERVAL%' THEN regexp_extract(delay, '([0-9]+:[0-9]+:[0-9]+)', 1) ELSE delay END"))
    df =  df.drop("time_arrival2")
    df =  df.drop("Flight Time2")
    # Concatenate month, day, and year columns to create a date string
    df = df.withColumn("date_string", concat(col("complete_month"), lit("-"), col("day_number"), lit("-"), col("year")))
    # Convert the date string to a date type
    df = df.withColumn("date", to_date(col("date_string"), "MMMM-dd-yyyy"))
    df=df.drop("date_string")

    return df


In [29]:
processed_departure_data = process_departure_data(departure_data)
processed_departure_data.toPandas()

C:\Users\USER\anaconda3333\lib\site-packages\pyspark\sql\column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


,date,Flight Time,Airline,Destination Airport,flight,Aircraft Model,Flight Status,time_arrival,Origin Airport,day,day_number,complete_month,Airport country,Destination Country,year,delay
0,2024-03-06,21:15,Wizz Air,"Bologna Guglielmo Marconi Airport, Italy",W43847,A320,Estimated dep.,21:30,Tirana International Airport,Wednesday,06,March,Albania,Italy,2024,00:15:00
1,2024-03-07,17:00,Aerolineas Argentinas (Retro Livery),"Formosa International Airport, Argentina",AR1760,B737,Scheduled,Scheduled,Buenos Aires Jorge Newbery Airport,Thursday,07,March,Argentina,Argentina,2024,Scheduled
2,2024-03-07,01:04,Copa Airlines,"Panama City Tocumen International Airport, Panama",CM501,738,Scheduled,Scheduled,Buenos Aires Ministro Pistarini International ...,Thursday,07,March,Argentina,Panama,2024,Scheduled
3,2024-03-07,06:50,Avianca,"Bogota El Dorado International Airport, Colombia",AV88,32N,Estimated dep.,06:50,Buenos Aires Ministro Pistarini International ...,Thursday,07,March,Argentina,Colombia,2024,00:00:00
4,2024-03-05,13:35,Aerolineas Argentinas,"Puerto Iguazu International Airport, Argentina",AR1731,E190,Estimated dep.,13:38,Salta Martin Miguel de Guemes International Ai...,Tuesday,05,March,Argentina,Argentina,2024,00:03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50086,2024-03-21,01:50,Jeju Air,"Seoul Incheon International Airport, South Korea",7C2804,B738,Departed,03:00,Hanoi Noi Bai International Airport,Thursday,21,March,Vietnam,South Korea,2024,01:10:00
50087,2024-03-21,09:35,VietJet Air,"Singapore Changi Airport, Singapore",VJ915,A321,Departed,10:07,Hanoi Noi Bai International Airport,Thursday,21,March,Vietnam,Singapore,2024,00:32:00
50088,2024-03-21,12:05,Vietnam Airlines,"Nha Trang Cam Ranh International Airport, Vietnam",VN1557,A321,Departed,12:22,Hanoi Noi Bai International Airport,Thursday,21,March,Vietnam,Vietnam,2024,00:17:00
50089,2024-03-23,01:40,VietJet Air,"Osaka Kansai International Airport, Japan",VJ938,321,Scheduled,Scheduled,Hanoi Noi Bai International Airport,Saturday,23,March,Vietnam,Japan,2024,Scheduled


In [83]:
column_types = processed_departure_data.dtypes

# Display the types
print(column_types)


[('date', 'date'), ('Flight Time', 'string'), ('Airline', 'string'), ('Destination Airport', 'string'), ('flight', 'string'), ('Aircraft Model', 'string'), ('Flight Status', 'string'), ('time_arrival', 'string'), ('Origin Airport', 'string'), ('day', 'string'), ('day_number', 'string'), ('complete_month', 'string'), ('Airport country', 'string'), ('Destination Country', 'string'), ('year', 'int'), ('delay', 'string')]


# Reviews data

In [30]:
# Example usage:
hdfs_url = 'http://localhost:50070'
folder_path = '/user/PFE_data/reviews_flights'  # Adjust this path accordingly

# Create an HDFS client
client = InsecureClient(hdfs_url)

# Get all CSV files from the specified folder and its subfolders
csv_files_dict = get_hdfs_csv_files(client, folder_path)
print()
# Process CSV files: open each CSV file, add a filename column, and store the DataFrame in a dictionary
dfs_reviews = process_csv_files(csv_files_dict, client)

dfs_reviews


Error parsing file Argentina2024-03-08.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 22

Empty file: Austria2024-03-14.csv
Empty file: Guernsey2024-03-14.csv
Error parsing file Latvia2024-03-07.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 22

Empty file: Malawi2024-03-14.csv
Empty file: Moldova2024-03-14.csv
Error parsing file Nigeria2024-03-07.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 22

Error parsing file Portugal2024-03-08.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 22

Empty file: Portugal2024-03-14.csv
Error parsing file Suriname2024-03-08.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 22

Error parsing file Ukraine2024-03-07.csv: Error tokenizing data. C error: Expected 11 fields in line 4, saw 22



,date,Stars Title,text review,Getting to the airport,Check-in,Security check,Terminal facilities,Food and retail services,WiFi,Lounge,Immigration/customs,Baggage claim,Destination Aeroport,Filename,{{objSubRating.subject}},Boarding/deplaning,Flight crew,In-Flight services,In-Flight entertainment
0,2024-03-08 07:30:31,Rated 4/5,"A very nice terminal, recent, modern, comforta...",Rated 2/5,Rated 5/5,Rated 5/5,Rated 5/5,Rated 4/5,NaN,NaN,NaN,NaN,Tirana International Airport,Albania2024-03-13.csv,NaN,NaN,NaN,NaN,NaN
1,2023-12-11 11:04:49,Rated 1/5,Lounge was closed without prior notice neither...,Rated 1/5,Rated 2/5,Rated 1/5,Rated 1/5,Rated 2/5,Rated 1/5,Rated 1/5,NaN,NaN,Tirana International Airport,Albania2024-03-13.csv,NaN,NaN,NaN,NaN,NaN
2,2023-10-13 04:50:22,Rated 3/5,In 2023 still not admit the digital boarding p...,Rated 3/5,Rated 2/5,Rated 3/5,Rated 3/5,NaN,Rated 3/5,NaN,NaN,NaN,Tirana International Airport,Albania2024-03-13.csv,NaN,NaN,NaN,NaN,NaN
3,2023-08-28 09:46:12,Rated 3/5,Tirana’s airport is way too small compared to ...,Rated 4/5,NaN,Rated 5/5,Rated 3/5,Rated 2/5,Rated 1/5,NaN,NaN,NaN,Tirana International Airport,Albania2024-03-13.csv,NaN,NaN,NaN,NaN,NaN
4,2022-06-01 13:47:01,Rated 3/5,The immigration staff are rude and impersonal....,Rated 2/5,Rated 5/5,Rated 3/5,Rated 2/5,Rated 3/5,Rated 1/5,Rated 3/5,NaN,NaN,Tirana International Airport,Albania2024-03-13.csv,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7205,2023-04-05 16:09:28,Rated 5/5,"Awesome airport, just the right size. Could do...",Rated 5/5,Rated 5/5,Rated 5/5,Rated 5/5,Rated 3/5,Rated 3/5,Rated 2/5,NaN,NaN,Ndola Simon Mwansa Kapwepwe International Airport,Zambia_reviews.csv,NaN,NaN,NaN,NaN,NaN
7206,2022-12-26 17:39:08,Rated 1/5,Airport was extremely hot and poorly lit. Was ...,Rated 2/5,NaN,NaN,Rated 1/5,NaN,NaN,NaN,Rated 2/5,Rated 1/5,Harare Robert Gabriel Mugabe International Air...,Zimbabwe2024-03-21.csv,NaN,NaN,NaN,NaN,NaN
7207,2021-11-30 16:18:12,Rated 3/5,I remember Harare being like an airport from t...,NaN,Rated 3/5,Rated 2/5,Rated 2/5,Rated 2/5,NaN,Rated 2/5,NaN,NaN,Harare Robert Gabriel Mugabe International Air...,Zimbabwe2024-03-21.csv,NaN,NaN,NaN,NaN,NaN
7208,2016-03-21 08:01:35,Rated 3/5,I used Hilton Transfers for a private transfer...,Rated 5/5,NaN,NaN,Rated 3/5,NaN,Rated 4/5,NaN,NaN,Rated 4/5,Harare Robert Gabriel Mugabe International Air...,Zimbabwe2024-03-21.csv,NaN,NaN,NaN,NaN,NaN


In [32]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Read from Hadoop") \
    .getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
#spark.conf.set("spark.memory.offHeap.enabled","true") 
#spark.conf.set("spark.memory.offHeap.size","10g")

# Set configurations

review_data = spark.createDataFrame(dfs_reviews)


In [80]:

from pyspark.sql.functions import col, split, regexp_extract, when
from pyspark.sql.functions import to_timestamp

def transform_departure_data(review_data):
    # Drop unnecessary columns
    columns_to_drop = ["Baggage claim", "Check-in", "In-Flight entertainment", 
                       "In-Flight services", "Flight crew", "Boarding/deplaning", 
                       "{{objSubRating.subject}}", "Immigration/customs"]
    df = review_data.drop(*columns_to_drop)

    # Define transformation for each column
    columns_to_transform = ["Getting to the airport", "Security check", "Terminal facilities",
                            "Food and retail services", "WiFi", "Stars Title", "Lounge"]
    for column in columns_to_transform:
        df = df.withColumn(column, col(column).substr(7, 1).cast("integer"))


    # Convert date column to timestamp format
    df = df.withColumn("time", to_timestamp("date", "yyyy-MM-dd HH:mm:ss"))

    # Extract Airport country from Filename column
    df = df.withColumn("Airport country",
                       when(df["Filename"].contains("_"),
                            split(col("Filename"), "_")[0])
                       .otherwise(regexp_extract(col("Filename"), "^(.*?)(?=[0-9])", 1)))
    df = df.drop("Filename")
    df = df.dropDuplicates()
   # df = df.withColumn("time", substring(col("date"), 12, 8))
    df = df.withColumn("date", substring(col("date"), 1, 10) )
    df = df.withColumn("date" ,to_date(df["date"], "yyyy-MM-dd"))
    df = df.withColumnRenamed("Destination Aeroport", "Airport")


    return df



In [81]:
# Apply transformation function to the departure_data DataFrame
transformed_review_data = transform_departure_data(review_data)

# Display the transformed DataFrame
transformed_review_data.toPandas()


,date,Stars Title,text review,Getting to the airport,Security check,Terminal facilities,Food and retail services,WiFi,Lounge,Airport,time,Airport country
0,2020-01-22,4.0,BAKU(GYD-UBBB)\nCheck-In-Desk:04-06\nGround.Ha...,4.0,4.0,4.0,4.0,4.0,4.0,Baku Heydar Aliyev International Airport,2020-01-22 05:16:38,Azerbaijan
1,2023-08-28,2.0,Domestic departures lounge was extremely crowd...,4.0,1.0,2.0,2.0,4.0,2.0,Buenos Aires Ministro Pistarini International ...,2023-08-28 18:38:20,Argentina
2,2021-08-31,5.0,We asked the lady at the check in for a window...,5.0,5.0,5.0,5.0,5.0,5.0,Tirana International Airport,2021-08-31 07:48:05,Albania
3,2015-01-04,5.0,El aeropuerto es muy lindo y comodo para llega...,5.0,4.0,3.0,4.0,1.0,4.0,Rio Grande International Airport,2015-01-04 20:46:39,Argentina
4,2022-08-01,2.0,"Very small airport, didn't have anything to do...",5.0,5.0,1.0,1.0,1.0,1.0,Banja Luka International Airport,2022-08-01 19:58:33,Bosnia-And-Herzegovina
...,...,...,...,...,...,...,...,...,...,...,...,...
6436,None,3.0,Very confusing layout and very busy. Long wait...,4.0,NaN,4.0,3.0,3.0,1.0,Malaga Costa Del Sol Airport,NaT,Spain
6437,None,4.0,Arrived on a Saturday evening in June only mem...,5.0,NaN,4.0,NaN,NaN,NaN,Malaga Costa Del Sol Airport,NaT,Spain
6438,2015-01-05,3.0,"A distant airport, some 50kms from town, PBM i...",3.0,NaN,3.0,3.0,1.0,0.0,Paramaribo Zanderij International Airport,2015-01-05 23:23:40,Suriname
6439,2021-12-26,4.0,Stopover in DAR. Smooth flight. Easy boarding....,4.0,NaN,2.0,NaN,NaN,NaN,Dar-es-Salaam Julius Nyerere International Air...,2021-12-26 05:44:48,Tanzania


In [82]:
column_types = transformed_review_data.dtypes

# Display the types
print(column_types)


[('date', 'date'), ('Stars Title', 'int'), ('text review', 'string'), ('Getting to the airport', 'int'), ('Security check', 'int'), ('Terminal facilities', 'int'), ('Food and retail services', 'int'), ('WiFi', 'int'), ('Lounge', 'int'), ('Airport', 'string'), ('time', 'timestamp'), ('Airport country', 'string')]


# Weather data

In [6]:
# Example usage:
hdfs_url = 'http://localhost:50070'
folder_path = '/user/PFE_data/weather_flights'  # Adjust this path accordingly

# Create an HDFS client
client = InsecureClient(hdfs_url)

# Get all CSV files from the specified folder and its subfolders
csv_files_dict = get_hdfs_csv_files(client, folder_path)
print()
# Process CSV files: open each CSV file, add a filename column, and store the DataFrame in a dictionary
dfs_weather = process_csv_files(csv_files_dict, client)

dfs_weather


Empty DataFrame found in file: Afghanistan2024-03-19.csv
Error parsing file American-Samoa2024-03-07.csv: Error tokenizing data. C error: Expected 4 fields in line 3, saw 10

Empty file: Bermuda2024-03-14.csv
Empty DataFrame found in file: Botswana_flights.csv
Error parsing file Brunei2024-03-07.csv: Error tokenizing data. C error: Expected 4 fields in line 3, saw 14

Error parsing file Djibouti2024-03-07.csv: Error tokenizing data. C error: Expected 4 fields in line 3, saw 14

Error parsing file Dominica2024-03-07.csv: Error tokenizing data. C error: Expected 4 fields in line 3, saw 14

Error parsing file Eswatini2024-03-07.csv: Error tokenizing data. C error: Expected 4 fields in line 3, saw 14

Error parsing file Greece2024-03-07.csv: Error tokenizing data. C error: Expected 4 fields in line 3, saw 14

Error parsing file Guam2024-03-07.csv: Error tokenizing data. C error: Expected 4 fields in line 3, saw 14

Error parsing file Ireland2024-03-07.csv: Error tokenizing data. C error: 

,Day,Time,Wind direction,Wind speed,Temperature,Dew point,Pressure,Visibility,date,Original Aeroport,Wind gust,Filename,d,a,t,e
0,6th,13:00 UTC,60°,14kt,25°C,1°C,1020 hPa,Sky and visibility OK,2024-03-06 13:00:00,Adrar Airport,NaN,Algeria_flights.csv,NaN,NaN,NaN,NaN
1,6th,11:00 UTC,60°,15kt,24°C,0°C,1020 hPa,Sky and visibility OK,2024-03-06 11:00:00,Adrar Airport,NaN,Algeria_flights.csv,NaN,NaN,NaN,NaN
2,6th,10:00 UTC,80°,18kt,23°C,-2°C,1020 hPa,Sky and visibility OK,2024-03-06 10:00:00,Adrar Airport,NaN,Algeria_flights.csv,NaN,NaN,NaN,NaN
3,6th,09:00 UTC,60°,14kt,18°C,-2°C,1020 hPa,Sky and visibility OK,2024-03-06 09:00:00,Adrar Airport,NaN,Algeria_flights.csv,NaN,NaN,NaN,NaN
4,6th,08:00 UTC,70°,11kt,14°C,-3°C,1020 hPa,Sky and visibility OK,2024-03-06 08:00:00,Adrar Airport,NaN,Algeria_flights.csv,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128869,3rd,14:00 UTC,NaN,4kt,23°C,7°C,1006 hPa,9999 m,2024-03-03 14:00:00,Tunis Carthage International Airport,NaN,tunisia_flights.csv,NaN,NaN,NaN,NaN
128870,3rd,13:30 UTC,180°,5kt,23°C,6°C,1006 hPa,9999 m,2024-03-03 13:30:00,Tunis Carthage International Airport,NaN,tunisia_flights.csv,NaN,NaN,NaN,NaN
128871,3rd,13:00 UTC,210°,6kt,24°C,7°C,1007 hPa,9999 m,2024-03-03 13:00:00,Tunis Carthage International Airport,NaN,tunisia_flights.csv,NaN,NaN,NaN,NaN
128872,3rd,12:30 UTC,180°,9kt,23°C,7°C,1007 hPa,9999 m,2024-03-03 12:30:00,Tunis Carthage International Airport,NaN,tunisia_flights.csv,NaN,NaN,NaN,NaN


In [7]:
dfs_weather

,Day,Time,Wind direction,Wind speed,Temperature,Dew point,Pressure,Visibility,date,Original Aeroport,Wind gust,Filename,d,a,t,e
0,6th,13:00 UTC,60°,14kt,25°C,1°C,1020 hPa,Sky and visibility OK,2024-03-06 13:00:00,Adrar Airport,NaN,Algeria_flights.csv,NaN,NaN,NaN,NaN
1,6th,11:00 UTC,60°,15kt,24°C,0°C,1020 hPa,Sky and visibility OK,2024-03-06 11:00:00,Adrar Airport,NaN,Algeria_flights.csv,NaN,NaN,NaN,NaN
2,6th,10:00 UTC,80°,18kt,23°C,-2°C,1020 hPa,Sky and visibility OK,2024-03-06 10:00:00,Adrar Airport,NaN,Algeria_flights.csv,NaN,NaN,NaN,NaN
3,6th,09:00 UTC,60°,14kt,18°C,-2°C,1020 hPa,Sky and visibility OK,2024-03-06 09:00:00,Adrar Airport,NaN,Algeria_flights.csv,NaN,NaN,NaN,NaN
4,6th,08:00 UTC,70°,11kt,14°C,-3°C,1020 hPa,Sky and visibility OK,2024-03-06 08:00:00,Adrar Airport,NaN,Algeria_flights.csv,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128869,3rd,14:00 UTC,NaN,4kt,23°C,7°C,1006 hPa,9999 m,2024-03-03 14:00:00,Tunis Carthage International Airport,NaN,tunisia_flights.csv,NaN,NaN,NaN,NaN
128870,3rd,13:30 UTC,180°,5kt,23°C,6°C,1006 hPa,9999 m,2024-03-03 13:30:00,Tunis Carthage International Airport,NaN,tunisia_flights.csv,NaN,NaN,NaN,NaN
128871,3rd,13:00 UTC,210°,6kt,24°C,7°C,1007 hPa,9999 m,2024-03-03 13:00:00,Tunis Carthage International Airport,NaN,tunisia_flights.csv,NaN,NaN,NaN,NaN
128872,3rd,12:30 UTC,180°,9kt,23°C,7°C,1007 hPa,9999 m,2024-03-03 12:30:00,Tunis Carthage International Airport,NaN,tunisia_flights.csv,NaN,NaN,NaN,NaN


In [8]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Read from Hadoop") \
    .getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

weather_data = spark.createDataFrame(dfs_weather)


In [10]:
from pyspark.sql.functions import when, create_map, lit, split, trim, regexp_extract, substring, concat, to_date, col

from itertools import chain
from pyspark.sql.functions import expr, when, col, unix_timestamp
from pyspark.sql.types import TimestampType

df = weather_data.withColumn("Country",
                       when(weather_data["Filename"].contains("_"),
                            split(col("Filename"), "_")[0])
                       .otherwise(regexp_extract(col("Filename"), "^(.*?)(?=[0-9])", 1)))
columns_to_drop = ["d", "a" , "t", "e" , "Wind gust" , "Filename"]
df = df.drop(*columns_to_drop)
df = df.dropDuplicates()


In [11]:
df.show()

+----+---------+--------------+----------+-----------+---------+--------+--------------------+-------------------+--------------------+---------+
| Day|     Time|Wind direction|Wind speed|Temperature|Dew point|Pressure|          Visibility|               date|   Original Aeroport|  Country|
+----+---------+--------------+----------+-----------+---------+--------+--------------------+-------------------+--------------------+---------+
| 6th|00:30 UTC|            0°|       0kt|       10°C|      9°C|1024 hPa|              8000 m|2024-03-06 00:30:00|Algiers Houari Bo...|  Algeria|
| 5th|15:00 UTC|          330°|      11kt|       18°C|     12°C|1023 hPa|              9999 m|2024-03-05 15:00:00|Algiers Houari Bo...|  Algeria|
| 5th|10:30 UTC|          310°|      11kt|       17°C|     10°C|1023 hPa|              9999 m|2024-03-05 10:30:00|Algiers Houari Bo...|  Algeria|
| 4th|15:00 UTC|          300°|      23kt|       16°C|      6°C|1014 hPa|              9999 m|2024-03-04 15:00:00|Annaba Rab

In [12]:
df.toPandas()

,Day,Time,Wind direction,Wind speed,Temperature,Dew point,Pressure,Visibility,date,Original Aeroport,Country
0,6th,00:30 UTC,0°,0kt,10°C,9°C,1024 hPa,8000 m,2024-03-06 00:30:00,Algiers Houari Boumediene Airport,Algeria
1,5th,15:00 UTC,330°,11kt,18°C,12°C,1023 hPa,9999 m,2024-03-05 15:00:00,Algiers Houari Boumediene Airport,Algeria
2,5th,10:30 UTC,310°,11kt,17°C,10°C,1023 hPa,9999 m,2024-03-05 10:30:00,Algiers Houari Boumediene Airport,Algeria
3,4th,15:00 UTC,300°,23kt,16°C,6°C,1014 hPa,9999 m,2024-03-04 15:00:00,Annaba Rabah Bitat Airport,Algeria
4,3rd,19:00 UTC,270°,11kt,12°C,4°C,1011 hPa,9999 m,2024-03-03 19:00:00,Annaba Rabah Bitat Airport,Algeria
...,...,...,...,...,...,...,...,...,...,...,...
126176,22nd,01:00 UTC,40°,5kt,28°C,21°C,1013 hPa,Sky and visibility OK,2024-03-22 01:00:00,Nha Trang Cam Ranh International Airport,Vietnam
126177,18th,07:53 UTC,90°,7kt,26°C,24°C,29.97 inHg,10 SM,2024-03-18 07:53:00,St. Thomas Cyril E. King Airport,Virgin-Islands-Us
126178,5th,10:30 UTC,260°,11kt,18°C,6°C,1021 hPa,Sky and visibility OK,2024-03-05 10:30:00,Djerba Zarzis International Airport,tunisia
126179,4th,12:00 UTC,310°,24kt,14°C,7°C,1014 hPa,9999 m,2024-03-04 12:00:00,Tabarka Ain Draham Airport,tunisia


In [13]:
from pyspark.sql.functions import regexp_replace, col

# Remove the degree symbol ('°') from the values in the "Wind direction" column
df = df.withColumn("Wind direction", regexp_replace(col("Wind direction"), "°", ""))

# Convert the column to integer type
df = df.withColumn("Wind direction", df["Wind direction"].cast("integer"))
# Rename column
df = df.withColumnRenamed("Wind direction", "Wind direction(°)")

median_value = df.approxQuantile("Wind direction(°)", [0.5], 0.001)[0]

# Replace NaN values in the specific column with the median
df = df.withColumn("Wind direction(°)", when(col("Wind direction(°)").isNull(), lit(median_value)).otherwise(col("Wind direction(°)")))

df = df.withColumn("Wind speed", regexp_replace(col("Wind speed"), "kt", ""))
# Convert the column to integer type
df = df.withColumn("Wind speed", df["Wind speed"].cast("integer"))
# Rename column
df = df.withColumnRenamed("Wind speed", "Wind speed(kt)")


df = df.withColumn("Temperature", regexp_replace(col("Temperature"), "°C", ""))
# Convert the column to integer type
df = df.withColumn("Temperature", df["Temperature"].cast("integer"))
# Rename column
df = df.withColumnRenamed("Temperature", "Temperature(°C)")


df = df.withColumn("Dew point", regexp_replace(col("Dew point"), "°C", ""))
# Convert the column to integer type
df = df.withColumn("Dew point", df["Dew point"].cast("integer"))
# Rename column
df = df.withColumnRenamed("Dew point", "Dew point(°C)")

df = df.withColumn("Pressure", regexp_replace(col("Pressure"), "hPa", ""))
# Convert the column to integer type
df = df.withColumn("Pressure", df["Pressure"].cast("integer"))
# Rename column
df = df.withColumnRenamed("Pressure", "Pressure(hPa)")

df = df.withColumn("Visibility", regexp_replace(col("Visibility"), "m", ""))
# Convert the column to integer type
df = df.withColumn("Visibility", df["Visibility"].cast("integer"))
# Rename column
df = df.withColumnRenamed("Visibility", "Visibility(m)")


# Show the result
df.toPandas()

,Day,Time,Wind direction(°),Wind speed(kt),Temperature(°C),Dew point(°C),Pressure(hPa),Visibility(m),date,Original Aeroport,Country
0,6th,00:30 UTC,0.0,0.0,10.0,9.0,1024.0,8000.0,2024-03-06 00:30:00,Algiers Houari Boumediene Airport,Algeria
1,5th,15:00 UTC,330.0,11.0,18.0,12.0,1023.0,9999.0,2024-03-05 15:00:00,Algiers Houari Boumediene Airport,Algeria
2,5th,10:30 UTC,310.0,11.0,17.0,10.0,1023.0,9999.0,2024-03-05 10:30:00,Algiers Houari Boumediene Airport,Algeria
3,4th,15:00 UTC,300.0,23.0,16.0,6.0,1014.0,9999.0,2024-03-04 15:00:00,Annaba Rabah Bitat Airport,Algeria
4,3rd,19:00 UTC,270.0,11.0,12.0,4.0,1011.0,9999.0,2024-03-03 19:00:00,Annaba Rabah Bitat Airport,Algeria
...,...,...,...,...,...,...,...,...,...,...,...
126176,22nd,01:00 UTC,40.0,5.0,28.0,21.0,1013.0,NaN,2024-03-22 01:00:00,Nha Trang Cam Ranh International Airport,Vietnam
126177,18th,07:53 UTC,90.0,7.0,26.0,24.0,NaN,NaN,2024-03-18 07:53:00,St. Thomas Cyril E. King Airport,Virgin-Islands-Us
126178,5th,10:30 UTC,260.0,11.0,18.0,6.0,1021.0,NaN,2024-03-05 10:30:00,Djerba Zarzis International Airport,tunisia
126179,4th,12:00 UTC,310.0,24.0,14.0,7.0,1014.0,9999.0,2024-03-04 12:00:00,Tabarka Ain Draham Airport,tunisia


In [20]:
panda = df.toPandas()[:1000]

In [23]:
import pandas as pd
import pytz
from datetime import datetime

# Supposons que vous ayez un DataFrame df avec les colonnes 'time' et 'country'

# Créer une fonction pour convertir l'heure en heure locale
def convert_to_local_time(row):
    country = row['Country']
    time_utc = row['Time']
    local_tz = pytz.country_timezones.get(country)
    if local_tz:
        local_tz = pytz.timezone(local_tz[0])
        time_local = time_utc.replace(tzinfo=pytz.utc).astimezone(local_tz)
        return time_local
    else:
        return None

# Appliquer la fonction à chaque ligne du DataFrame
panda['time_local'] = panda.apply(convert_to_local_time, axis=1)

# Afficher le DataFrame avec la nouvelle colonne time_local
panda


,Day,Time,Wind direction(°),Wind speed(kt),Temperature(°C),Dew point(°C),Pressure(hPa),Visibility(m),date,Original Aeroport,Country,time_local
0,6th,00:30 UTC,0.0,0.0,10.0,9.0,1024.0,8000.0,2024-03-06 00:30:00,Algiers Houari Boumediene Airport,Algeria,None
1,5th,15:00 UTC,330.0,11.0,18.0,12.0,1023.0,9999.0,2024-03-05 15:00:00,Algiers Houari Boumediene Airport,Algeria,None
2,5th,10:30 UTC,310.0,11.0,17.0,10.0,1023.0,9999.0,2024-03-05 10:30:00,Algiers Houari Boumediene Airport,Algeria,None
3,4th,15:00 UTC,300.0,23.0,16.0,6.0,1014.0,9999.0,2024-03-04 15:00:00,Annaba Rabah Bitat Airport,Algeria,None
4,3rd,19:00 UTC,270.0,11.0,12.0,4.0,1011.0,9999.0,2024-03-03 19:00:00,Annaba Rabah Bitat Airport,Algeria,None
...,...,...,...,...,...,...,...,...,...,...,...,...
995,3rd,14:00 UTC,360.0,3.0,14.0,7.0,1037.0,9999.0,2024-03-03 14:00:00,Oruro Juan Mendoza Airport,Bolivia,None
996,3rd,12:00 UTC,350.0,12.0,27.0,24.0,1012.0,9999.0,2024-03-03 12:00:00,Santa Cruz Viru Viru International Airport,Bolivia,None
997,17th,21:00 UTC,100.0,5.0,5.0,5.0,1018.0,6000.0,2024-03-17 21:00:00,Varna Airport,Bulgaria,None
998,16th,20:30 UTC,340.0,2.0,4.0,3.0,1021.0,8000.0,2024-03-16 20:30:00,Varna Airport,Bulgaria,None


In [82]:
row = panda.loc[0]
print(row)
country = row['Country']
print(country)
time_utc = row['date']
print(time_utc)
time_utc = datetime.strptime(time_utc, "%H:%M %Z")


print(time_utc)
local_tz = pytz.country_timezones.get("Algeria")
print(local_tz)

Day                                                6th
Time                                         00:30 UTC
Wind direction(°)                                  0.0
Wind speed(kt)                                     0.0
Temperature(°C)                                   10.0
Dew point(°C)                                      9.0
Pressure(hPa)                                   1024.0
Visibility(m)                                   8000.0
date                               2024-03-06 00:30:00
Original Aeroport    Algiers Houari Boumediene Airport
Country                                        Algeria
time_local                                        None
Name: 0, dtype: object
Algeria
2024-03-06 00:30:00


ValueError: time data '2024-03-06 00:30:00' does not match format '%H:%M %Z'

In [80]:

print(time_utc)

2024-01-01 00:30:00


In [81]:
import pycountry

country_code = pycountry.countries.get(name="Tunisia").alpha_2
local_tz = pytz.country_timezones.get(country_code)
local_tzinfo = pytz.timezone(local_tz[0])
    # Convert the datetime from UTC to the local time of the country
time_local = time_utc.replace(tzinfo=pytz.utc).astimezone(local_tzinfo)

time_local.strftime("%Y-%m-%d %H:%M:%S %Z")


TN
['Africa/Tunis']
Africa/Tunis


'2024-01-01 01:30:00 CET'

In [70]:
local_tz

In [69]:
local_tz = pytz.country_timezones.get(country)


<DstTzInfo 'Africa/Algiers' LMT+0:12:00 STD>

In [67]:
time_local = time_utc.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return time_local.strftime("%Y-%m-%d %H:%M:%S %Z")


datetime.datetime(1900, 1, 1, 0, 42, tzinfo=<DstTzInfo 'Africa/Algiers' LMT+0:12:00 STD>)

'1900-01-01 00:42:00 LMT'

datetime.datetime(1900, 1, 1, 0, 42, tzinfo=<DstTzInfo 'Africa/Algiers' LMT+0:12:00 STD>)

In [84]:
df.select("date").show()

+-------------------+
|               date|
+-------------------+
|2024-03-06 00:30:00|
|2024-03-05 15:00:00|
|2024-03-05 10:30:00|
|2024-03-04 15:00:00|
|2024-03-03 19:00:00|
|2024-03-04 02:00:00|
|2024-03-04 14:00:00|
|2024-03-05 16:00:00|
|2024-03-06 07:00:00|
|2024-03-04 00:00:00|
|2024-03-06 01:00:00|
|2024-03-05 13:00:00|
|2024-03-03 19:30:00|
|2024-03-04 11:00:00|
|2024-03-20 17:00:00|
|2024-03-21 11:00:00|
|2024-03-21 03:00:00|
|2024-03-04 07:00:00|
|2024-03-02 16:00:00|
|2024-03-02 02:00:00|
+-------------------+
only showing top 20 rows

